# 1. Use Naive Bayes to Classify Spam Message

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

### 1.1 Data
- 문자 내용이 스팸인지 아닌지를 구분하기 위한 데이터

In [2]:
spam = pd.read_csv("../02. Data/sms_spam.csv")

text = spam["text"]
label = spam["type"]

### 1.2 EDA

In [3]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [4]:
label[0]

'ham'

In [5]:
label.value_counts()

ham     4827
spam     747
Name: type, dtype: int64

### 1.3 Data Cleaning
- 정답의 문자를 숫자로 변환 -> ham은 0으로, spam은 1로 변환
- text를 문자만 존재하도록 정리
- 대문자들을 모두 소문자로 변환

In [6]:
label = label.map({"ham": 0, "spam": 1})

label.value_counts()

0    4827
1     747
Name: type, dtype: int64

In [7]:
re_pattern = "[^a-zA-Z0-9\ ]"

# 영어,숫자, 띄어쓰기 제외 문자 제거전
print(text[0])

# 문자 제거후
text = text.str.replace(re_pattern, "", regex=True)
print(text[0])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat


In [8]:
# 소문자 변환전
print(text[0])

# 소문자 변환후
text = text.str.lower()
print(text[0])

Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat


### 1.4 Data Split

In [9]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(
    text, label, train_size=0.7, random_state=2021)

print(f"train_data size: {len(train_label)}, {len(train_label)/len(text):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(text):.2f}")

train_data size: 3901, 0.70
test_data size: 1673, 0.30


# 2. Count Vectorize
- Naive Bayes를 학습시키기 위해서 각 문장에서 단어들이 몇 번 나왔는지로 변환

### 2.1 word tokenize
- 문장을 단어로 나누는 데에는 `nltk` 패키지의 `word_tokenize`를 이용

In [10]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\inp01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
print(train_text.iloc[0])
print(word_tokenize(train_text.iloc[0]))

am only searching for good dual sim mobile pa
['am', 'only', 'searching', 'for', 'good', 'dual', 'sim', 'mobile', 'pa']


### 2.2 count vectorize
- `CountVectorizer`를 이용해 단어들을 count vector로 생성

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [13]:
# 예시로 2개 문장 CountVectorizer를 학습
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text.iloc[:2])

C:\Users\inp01\Anaconda3\envs\ojt\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x000001DDD217B0D0>)

In [14]:
cnt_vectorizer.vocabulary_

{'am': 0,
 'only': 8,
 'searching': 12,
 'for': 4,
 'good': 5,
 'dual': 2,
 'sim': 14,
 'mobile': 7,
 'pa': 9,
 'excellent': 3,
 'ill': 6,
 'see': 13,
 'what': 15,
 'rileys': 11,
 'plans': 10,
 'are': 1}

In [15]:
vocab = sorted(cnt_vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab = list(map(lambda x: x[0], vocab))
vocab

['am',
 'are',
 'dual',
 'excellent',
 'for',
 'good',
 'ill',
 'mobile',
 'only',
 'pa',
 'plans',
 'rileys',
 'searching',
 'see',
 'sim',
 'what']

In [16]:
sample_cnt_vector = cnt_vectorizer.transform(train_text.iloc[:2]).toarray()
sample_cnt_vector

array([[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1]], dtype=int64)

In [17]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [18]:
pd.DataFrame(sample_cnt_vector, columns=vocab)

,am,are,dual,excellent,for,good,ill,mobile,only,pa,plans,rileys,searching,see,sim,what
0,1,0,1,0,1,1,0,1,1,1,0,0,1,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,1


#### 2.2.1 Learning

In [19]:
# 모든 데이터 진행
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text)

C:\Users\inp01\Anaconda3\envs\ojt\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x000001DDD217B0D0>)

In [20]:
print('전체 단어 개수 : {}'.format(len(cnt_vectorizer.vocabulary_)))

전체 단어 개수 : 7908


#### 2.2.2 Predict
- CountVectorize는 학습한 단어장에 존재하지 않는 단어가 들어오게 될 경우 무시

In [21]:
train_matrix = cnt_vectorizer.transform(train_text)
test_matrix = cnt_vectorizer.transform(test_text)

cnt_vectorizer.transform(["notavailblewordforcnt"]).toarray().sum()

0

# 3. Naive Bayes
- 분류를 위한 Naive Bayes 모델은 `sklearn.naive_bayes`의 `BernoulliNB`를 사용

In [22]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()

### 3.1 Learning

In [23]:
naive_bayes.fit(train_matrix, train_label)

BernoulliNB()

### 3.2 Predict

In [24]:
train_pred = naive_bayes.predict(train_matrix)
test_pred = naive_bayes.predict(test_matrix)

### 3.3 Evalution

In [25]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

print(f"Train Accuracy is {train_acc:.4f}")
print(f"Test Accuracy is {test_acc:.4f}")

Train Accuracy is 0.9854
Test Accuracy is 0.9767
